In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
# Helper
df = pd.read_csv("", error_bad_lines=False)
df.head()

In [ ]:
# Helper
for run in df.groupby('run'):
    df = run[1]
    print("Run {} went {} days w/ {} Ixodes".format(run[0], df['tick'].max(), df['name'].value_counts().count()))

In [ ]:
#
# Parses parameter file and creates a dictionary that maps the habitat_suitability value with the
# corresponding run_number and host_density values
#
# { habitat: [ [run_number], [host_density] ] }
#
paramfile = ""
paramd = {}
with open(paramfile, 'r') as file:
    for line in file:
        result = line.replace("\t",",").split(',')
# Try to add the values to an existing key, if that key does not exist, create it
        try:
            paramd[float(result[8])][0].append(int(result[0])) # add run number
            paramd[float(result[8])][1].append(float(result[4])) # add host density
        except KeyError:
            paramd[float(result[8])] = [[int(result[0])], [float(result[4])]]
# Helper
for k, v in paramd.items():
    print(k,v)


In [ ]:
ixode_count_dict = {}
df = pd.read_csv("", error_bad_lines=False)
for run in df.groupby('run'):
    current_df = run[1]
    ixode_count_dict[run[0]] = len(current_df['name'].unique())

In [ ]:
# Creating a data frame from the above generated dictionary
df_final = pd.DataFrame(ixode_count_dict.items(), columns=['run', 'total_ixode'])
df_final['host_density'] = 0
df_final.dtypes

In [ ]:
# Iterate through parameter dictionary and use loc to find the row with the run number we want. Once we have this, add
# the host density value for that associated run
for key, value in paramd.items():
    for i in range(len(value[0])):
        # Both of these techniques work, but loc is preferred over chained indexing
        # df_final['host_density'][df_final['run'] == value[0][i]] = value[1][i]
        df_final.loc[df_final['run']==value[0][i], 'host_density'] = value[1][i]
        df_final.loc[df_final['run']==value[0][i], 'habitat_suitability'] = key

In [ ]:
matplotlib.use('Qt5Agg')
# fig = plt.figure()
for hab in df_final.groupby('habitat_suitability'):
    dat = hab[1]
    plt.plot('host_density', 'total_ixode' , data=dat, marker='o', markersize=7, linewidth=2, label=hab[0])
    plt.ylabel('Cumulative Ixodes')
    plt.xlabel('Large host density')
    plt.title("Host Density")
    plt.legend() # FIX to show values not just label
plt.show(block=True)
# plt.savefig("../data/host-density/plots/newmodel-density-adult-10ixodes-labeled.png")

